In [1]:
%tensorflow_version 1.x

TensorFlow 1.x selected.


In [2]:
# Load the TensorBoard notebook extension.
%load_ext tensorboard

In [3]:
import tensorflow
print(tensorflow.__version__)
import tensorboard
print(tensorboard.__version__)

1.15.2
1.15.0


In [12]:
frozen_graph = 'opencv_face_detector_uint8.pb'
output_graph = "frozen_inference_graph_dnn.pb"
LOGDIR = '/content/logs'
# Clear any logs from previous runs
%cd /content/
!rm -rf ./logs/ 

/content


In [29]:
import tensorflow as tf
from tensorflow.tools.graph_transforms import TransformGraph
from tensorflow.core.framework import graph_pb2
import copy
import cv2
import numpy as np

In [16]:
def load_graph(filename):
    print(f'load_graph {filename}')
    graph_def = tf.GraphDef()
    with tf.gfile.FastGFile(filename, 'rb') as f:
        graph_def.ParseFromString(f.read())
    return graph_def


In [21]:
def transform_graph(input_graph, output_graph=None):
    """ Use to run different transform function on the input graph and generate a output graph. """
    print(input_graph)
    if isinstance(input_graph, graph_pb2.GraphDef):
        graph_def = input_graph
    else:
        graph_def = load_graph(input_graph)

    print("swa")
    # new_graph_def = TransformGraph(graph_def, ['input_placeholder/input_image'], ['predicated_output'],
    #                                ['strip_unused_nodes(type=float, shape="1,28,28,1")',
    #                                 'remove_nodes(op=Identity, op=CheckNumerics, op=Switch)',
    #                                 'fold_constants(ignore_errors=true)', 'fold_batch_norms', 'fold_old_batch_norms',
    #                                 'sort_by_execution_order'])
    new_graph_def = TransformGraph(graph_def, ['input_placeholder/input_image'], [],
                                   ['strip_unused_nodes(type=float, shape="1,28,28,1")',
                                    'remove_nodes(op=Identity, op=CheckNumerics, op=Switch)',
                                    'fold_constants(ignore_errors=true)', 'fold_batch_norms', 'fold_old_batch_norms',
                                    'sort_by_execution_order'])

    if output_graph == None:
        return new_graph_def

    # save new graph
    with tf.gfile.GFile(output_graph, "wb") as f:
        f.write(new_graph_def.SerializeToString())

    # tf.io.write_graph(od_graph_def, "", output_graph, as_text=False)



In [8]:
def convert_to_constant(input_graph, output_graph=None):
    """ Convert the placeholders in graph to constant nodes. """
    if isinstance(input_graph, graph_pb2.GraphDef):
        graph_def = input_graph
    else:
        graph_def = load_graph(input_graph)

    keep_prob = tf.constant(1.0, dtype=tf.float32, shape=[], name='keep_prob')
    weight_factor = tf.constant(1.0, dtype=tf.float32, shape=[], name='weight_factor')
    is_training = tf.constant(False, dtype=tf.bool, shape=[], name='is_training')

    new_graph_def = graph_pb2.GraphDef()

    for node in graph_def.node:
        if node.name == 'keep_prob':
            new_graph_def.node.extend([keep_prob.op.node_def])

        elif node.name == 'weight_factor':
            new_graph_def.node.extend([weight_factor.op.node_def])

        elif node.name == 'is_training':
            new_graph_def.node.extend([is_training.op.node_def])

        else:
            new_graph_def.node.extend([copy.deepcopy(node)])

    if output_graph == None:
        return new_graph_def

    # save new graph
    with tf.gfile.GFile(output_graph, "wb") as f:
        f.write(new_graph_def.SerializeToString())


In [9]:
def optimize_batch_normalization(input_graph, output_graph=None):
    """ Optimize the batch normalization block. """
    if isinstance(input_graph, graph_pb2.GraphDef):
        graph_def = input_graph
    else:
        graph_def = load_graph(input_graph)

    new_graph_def = graph_pb2.GraphDef()
    unused_attrs = ['is_training']  # Attributes of FusedBatchNorm. Not needed during inference.

    # All the node names are specific to my ocr model.
    # All the input names are found manually from tensorboard
    for node in graph_def.node:
        modified_node = copy.deepcopy(node)
        if node.name.startswith("conv"):  # True for Convolutional Layers
            starting_name = ""
            if node.name.startswith("conv1"):
                starting_name = "conv1"

            elif node.name.startswith("conv2"):
                starting_name = "conv2"

            elif node.name.startswith("conv3"):
                starting_name = "conv3"

            elif node.name.startswith("conv4"):
                starting_name = "conv4"

            # Do not add the cond block and its child nodes.
            # This is only needed during training.
            if "cond" in node.name and not node.name.endswith("FusedBatchNorm"):
                continue

            if node.op == "FusedBatchNorm" and node.name.endswith("FusedBatchNorm"):
                if bool(starting_name):
                    # Changing the name to remove one block hierarchy and changing inputs.
                    modified_node.name = "{0}/{0}/batch_norm/FusedBatchNorm".format(starting_name)
                    modified_node.input[0] = "{}/Conv2D".format(starting_name)
                    modified_node.input[1] = "{}/batch_norm/gamma".format(starting_name)
                    modified_node.input[2] = "{}/batch_norm/beta".format(starting_name)
                    modified_node.input[3] = "{}/batch_norm/moving_mean".format(starting_name)
                    modified_node.input[4] = "{}/batch_norm/moving_variance".format(starting_name)

                    # Deleting unused attributes
                    for attr in unused_attrs:
                        if attr in modified_node.attr:
                            del modified_node.attr[attr]

            if node.name.endswith('activation'):
                if bool(starting_name):
                    modified_node.input[0] = "{0}/{0}/batch_norm/FusedBatchNorm".format(starting_name)

        elif node.name.startswith("fc") or node.name.startswith("logits"):  # True for fully connected layers
            starting_name = ""
            if node.name.startswith("fc1"):
                starting_name = "fc1"

            elif node.name.startswith("fc2"):
                starting_name = "fc2"

            elif node.name.startswith("logits"):
                starting_name = "logits"

            # Do not add cond, cond_1 and moments block of batch normalization
            if "cond" in node.name or "moments" in node.name:
                continue

            # Change input of batchnorm/add
            if node.name.endswith('batchnorm/add'):
                modified_node.input[0] = "{}/batch_norm/moving_variance".format(starting_name)
                modified_node.input[1] = "{0}/{0}/batch_norm/batchnorm/add/y".format(starting_name)

            if node.name.endswith('batchnorm/mul_2'):
                modified_node.input[0] = "{0}/{0}/batch_norm/batchnorm/mul".format(starting_name)
                modified_node.input[1] = "{}/batch_norm/moving_mean".format(starting_name)

        new_graph_def.node.extend([modified_node])

    if output_graph == None:
        return new_graph_def

    # save the graph
    with tf.gfile.GFile(output_graph, "wb") as f:
        f.write(new_graph_def.SerializeToString())

In [10]:
def remove_dropout(input_graph, output_graph=None):
    """ Remove the dropout block from the model. """
    if isinstance(input_graph, graph_pb2.GraphDef):
        graph_def = input_graph
    else:
        graph_def = load_graph(input_graph)

    new_graph_def = graph_pb2.GraphDef()

    for node in graph_def.node:
        modified_node = copy.deepcopy(node)
        if node.name.startswith('dropout1') or node.name.startswith('dropout2'):
            continue

        if node.name == "fc2/fc2/batch_norm/batchnorm/mul_1":
            modified_node.input[0] = "mul"
            modified_node.input[1] = "fc2/weights"

        if node.name == "logits/logits/batch_norm/batchnorm/mul_1":
            modified_node.input[0] = "fc2/activation"
            modified_node.input[1] = "logits/weights"

        new_graph_def.node.extend([modified_node])

    if output_graph == None:
        return new_graph_def

    # save the graph
    with tf.gfile.GFile(output_graph, "wb") as f:
        f.write(new_graph_def.SerializeToString())


In [22]:
graph_def = transform_graph(frozen_graph)

opencv_face_detector_uint8.pb
load_graph opencv_face_detector_uint8.pb
swa


In [23]:
graph_def = convert_to_constant(graph_def)


In [24]:
graph_def = optimize_batch_normalization(graph_def)

In [25]:
graph_def = transform_graph(graph_def)


swa


In [26]:
remove_dropout(graph_def, output_graph)

In [31]:
def test_dnn_module():
    """ Function to test OpenCV DNN module. """
    model = 'frozen_inference_graph_dnn.pb'
    net = cv2.dnn.readNet(model)
    img = cv2.imread('tv24horas_2022_01_06_10_frame_124425.png')
    imgGray = cv2.cvtColor(img, cv2.COLOR_BGR2GRAY)
    imgGray = imgGray.astype(np.uint8)
    blob = cv2.dnn.blobFromImage(imgGray, 1.0, (28,28), swapRB=True, crop=False)
    net.setInput(blob)
    output = net.forward()
    print(output)

In [ ]:
test_dnn_module()